### 네이버 리뷰 감성 분석
- 한글 텍스트 분석 : KoNLpy ( 자바모듈로 기반 파이썬 래퍼 패키지)
- https://konlpy.org/ko/latest/   : Mecap 이 제일 수행속도가 빠름
- 데이터 셋 :  https://github.com/e9t/nsmc

# Naver sentiment movie corpus v1.0

This is a movie review dataset in the Korean language.
Reviews were scraped from [Naver Movies](http://movie.naver.com/movie/point/af/list.nhn).

The dataset construction is based on the method noted in [Large movie review dataset](http://ai.stanford.edu/~amaas/data/sentiment/) from Maas et al., 2011.


## Data description

- Each file is consisted of three columns: `id`, `document`, `label`
    - `id`: The review id, provieded by Naver
    - `document`: The actual review
    - `label`: The sentiment class of the review. (0: negative, 1: positive)
    - Columns are delimited with tabs (i.e., `.tsv` format; but the file extension is `.txt` for easy access for novices)
- 200K reviews in total
    - `ratings.txt`: All 200K reviews
    - `ratings_test.txt`: 50K reviews held out for testing
    - `ratings_train.txt`: 150K reviews for training

## Characteristics

- All reviews are shorter than 140 characters
- Each sentiment class is sampled equally (i.e., random guess yields 50% accuracy)
    - 100K negative reviews (originally reviews of ratings 1-4)
    - 100K positive reviews (originally reviews of ratings 9-10)
    - Neutral reviews (originally reviews of ratings 5-8) are excluded

## Quick peek

    $ head ratings_train.txt
    id      document        label
    9976970 아 더빙.. 진짜 짜증나네요 목소리        0
    3819312 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나        1
    10265843        너무재밓었다그래서보는것을추천한다      0
    9045019 교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정       0
    6483659 사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다  1
    5403919 막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.     0
    7797314 원작의 긴장감을 제대로 살려내지못했다.  0
    9443947 별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네       0
    7156791 액션이 없는데도 재미 있는 몇안되는 영화 1

## License

<p xmlns:dct="http://purl.org/dc/terms/">
  <a rel="license"
     href="http://creativecommons.org/publicdomain/zero/1.0/">
    <img src="http://i.creativecommons.org/p/zero/1.0/88x31.png" style="border-style: none;" alt="CC0" />
  </a>
</p>


In [1]:
import numpy as np
import pandas as pd

In [2]:
import konlpy

In [31]:
train_df = pd.read_csv('./data/nsmc-master/ratings_train.txt', sep='\t')

In [32]:
train_df

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [33]:
# 레이블의 분포 확인
train_df['label'].value_counts()

0    75173
1    74827
Name: label, dtype: int64

- 균형잡힌 데이터셋이다

In [34]:
# Null 값의 개수 확인
train_df['document'].isnull().sum()

5

In [35]:
# Null 인 애들의 타입을 object(str) 로 바꿔주기 ( 나중에 연산상의 문제가 없도록 하기 위해)
train_df = train_df.fillna(' ')

In [36]:
train_df['document'].isnull().sum()

0

In [39]:
# 리뷰에 있는 숫자형식의 문자를 공백으로 바꾸기(의미없는 숫자의 제거) : [0-9]+ == \d+
import re
train_df['document'] = train_df['document'].apply(lambda x : re.sub(r'\d+', ' ', x))

In [40]:
train_df['document'][:6]

0                                  아 더빙.. 진짜 짜증나네요 목소리
1                    흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2                                    너무재밓었다그래서보는것을추천한다
3                        교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
4    사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...
5        막 걸음마 뗀  세부터 초등학교  학년생인  살용영화.ㅋㅋㅋ...별반개도 아까움.
Name: document, dtype: object

In [41]:
# id 컬럼 삭제
train_df.drop('id', axis=1, inplace=True)

In [45]:
# 형태소 분석 연습
from konlpy.tag import Okt

In [46]:
twitter = Okt()
def tw_tokenizer(text):
    tokens_ko = twitter.morphs(text)
    return tokens_ko

In [47]:
tw_tokenizer('아버지가 방에 들어가신다')

['아버지', '가', '방', '에', '들어가신다']

In [48]:
tw_tokenizer('아버지 가방에 들어가신다')

['아버지', '가방', '에', '들어가신다']

In [49]:
tw_tokenizer('너를')

['너', '를']

In [50]:
tw_tokenizer('머리결')

['머리', '결']

In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [52]:
# Twitter 객체의 morphs( ) 객체를 이용한 tokenizer를 사용. ngram_range는 (1,2) 
tfidf_vect = TfidfVectorizer(tokenizer=tw_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf_vect.fit(train_df['document'])
tfidf_matrix_train = tfidf_vect.transform(train_df['document'])

C:\Users\TJ\anaconda3\envs\ml01-02\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [53]:
# 테스트 데이터 셋을 로딩하고 동일하게 Null 및 숫자를 공백으로 변환
test_df = pd.read_csv('./data/nsmc-master/ratings_test.txt', sep='\t')
test_df = test_df.fillna(' ')
test_df['document'] = test_df['document'].apply( lambda x : re.sub(r"\d+", " ", x) )
test_df.drop('id', axis=1, inplace=True)

In [54]:
# Logistic Regression 을 이용하여 감성 분석 Classification 수행. 
lg_clf = LogisticRegression(random_state=0)

In [55]:
# Parameter C 최적화를 위해 GridSearchCV 를 이용. 
params = { 'C': [1 ,3.5, 4.5, 5.5, 10 ] }
grid_cv = GridSearchCV(lg_clf , param_grid=params , cv=3 ,scoring='accuracy', verbose=1 )

In [56]:
grid_cv.fit(tfidf_matrix_train , train_df['label'] )
print(grid_cv.best_params_ , round(grid_cv.best_score_,4))

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\TJ\anaconda3\envs\ml01-02\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\TJ\anaconda3\envs\ml01-02\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for a

{'C': 3.5} 0.8592


C:\Users\TJ\anaconda3\envs\ml01-02\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [57]:
from sklearn.metrics import accuracy_score

In [58]:
# 학습 데이터를 적용한 TfidfVectorizer를 이용하여 테스트 데이터를 TF-IDF 값으로 Feature 변환함. 
tfidf_matrix_test = tfidf_vect.transform(test_df['document'])

In [59]:
# classifier 는 GridSearchCV에서 최적 파라미터로 학습된 classifier를 그대로 이용
best_estimator = grid_cv.best_estimator_
preds = best_estimator.predict(tfidf_matrix_test)

print('Logistic Regression 정확도: ',accuracy_score(test_df['label'],preds))

Logistic Regression 정확도:  0.86186


In [60]:
# Mecab 구동 여부 확인
from konlpy.tag import Mecab


In [62]:
mecab = Mecab()
def tw_tokenizer_(text):
    tokens_ko = mecab.morphs(text)
    return tokens_ko

Exception: Install MeCab in order to use it: http://konlpy.org/en/latest/install/

- 윈도우에서는 구동되지 않는다.